In [34]:
import os
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

os.makedirs("data", exist_ok=True)
os.makedirs("reports", exist_ok=True)

RAW_CSV_PATH = "data/intraday.csv"
DB_PATH = "data/market.db"
TABLE = "bars_1min"


In [35]:
import pandas as pd
import os

os.makedirs("reports", exist_ok=True)

# Load your trades_df from your notebook (must already exist)
# If your variable name is different, replace trades_df below with your name.

trades_df.to_csv("reports/trades.csv", index=False)

daily = equity.copy()
daily["date"] = pd.to_datetime(daily["datetime"]).dt.date
daily = daily.groupby("date")["equity"].agg(["first", "last"])
daily["daily_return"] = (daily["last"]/daily["first"]) - 1
daily = daily.reset_index()

# If you renamed last->equity earlier, do it again:
daily = daily.rename(columns={"last": "equity"})
daily.to_csv("reports/daily_summary.csv", index=False)

print("✅ Saved reports/trades.csv and reports/daily_summary.csv")


✅ Saved reports/trades.csv and reports/daily_summary.csv


In [36]:
import pandas as pd

daily = pd.read_csv("reports/daily_summary.csv")
daily = daily.rename(columns={"last": "equity"})
daily.to_csv("reports/daily_summary.csv", index=False)

daily.head()


,date,first,equity,daily_return
0,2025-12-01,1.000000,0.999553,-0.000447
1,2025-12-02,0.999553,0.998789,-0.000764
2,2025-12-03,0.998789,0.998517,-0.000273
3,2025-12-04,0.998517,0.995278,-0.003244
4,2025-12-05,0.995278,0.994611,-0.000670


In [37]:
np.random.seed(42)

def generate_intraday(start_date="2025-12-01", days=5):
    start = pd.to_datetime(start_date)
    all_rows = []

    for d in range(days):
        day = start + pd.Timedelta(days=d)

        # Market session: 9:15 to 15:30 (375 minutes)
        session_start = day + pd.Timedelta(hours=9, minutes=15)
        n = 375
        times = pd.date_range(session_start, periods=n, freq="1min")

        price = 45000 + np.cumsum(np.random.normal(0, 8, size=n))
        spread = np.random.uniform(2, 10, size=n)

        o = price + np.random.normal(0, 2, size=n)
        c = price + np.random.normal(0, 2, size=n)
        h = np.maximum(o, c) + spread
        l = np.minimum(o, c) - spread
        v = np.random.randint(200, 2500, size=n)

        df = pd.DataFrame({
            "datetime": times,
            "open": o,
            "high": h,
            "low": l,
            "close": c,
            "volume": v
        })
        all_rows.append(df)

    return pd.concat(all_rows, ignore_index=True)

df = generate_intraday()
df.to_csv(RAW_CSV_PATH, index=False)
df.head()


,datetime,open,high,low,close,volume
0,2025-12-01 09:15:00,45002.439018,45011.713948,44995.764504,45005.039433,1449
1,2025-12-01 09:16:00,45005.644763,45014.854027,44991.150718,45000.359982,722
2,2025-12-01 09:17:00,45002.730404,45013.166285,45000.366833,45010.802714,721
3,2025-12-01 09:18:00,45016.076593,45026.035084,45011.828887,45021.787378,982
4,2025-12-01 09:19:00,45015.878839,45028.766662,45006.275547,45019.163370,1135


In [38]:
df = pd.read_csv(RAW_CSV_PATH)
df.columns = [c.strip().lower() for c in df.columns]

df["datetime"] = pd.to_datetime(df["datetime"], errors="coerce")
df = df.dropna(subset=["datetime"])

for col in ["open","high","low","close","volume"]:
    df[col] = pd.to_numeric(df[col], errors="coerce")
df = df.dropna(subset=["open","high","low","close","volume"])

df = df.sort_values("datetime").drop_duplicates(subset=["datetime"]).reset_index(drop=True)

engine = create_engine(f"sqlite:///{DB_PATH}")
df.to_sql(TABLE, engine, if_exists="replace", index=False)

print("✅ Stored rows in SQL:", len(df))


✅ Stored rows in SQL: 1875


In [39]:
FAST_MA = 20
SLOW_MA = 50

d = df.copy()
d["date"] = d["datetime"].dt.date
d["tp"] = (d["high"] + d["low"] + d["close"]) / 3

d["cum_vol"] = d.groupby("date")["volume"].cumsum()
d["cum_vp"] = (d["tp"] * d["volume"]).groupby(d["date"]).cumsum()
d["vwap"] = d["cum_vp"] / d["cum_vol"].replace(0, np.nan)

d["ma_fast"] = d["close"].rolling(FAST_MA).mean()
d["ma_slow"] = d["close"].rolling(SLOW_MA).mean()

d["delta"] = np.sign(d["close"].diff().fillna(0)) * d["volume"]
d["delta_rolling"] = d["delta"].rolling(20).sum()

d.tail()


,datetime,open,high,low,close,volume,date,tp,cum_vol,cum_vp,vwap,ma_fast,ma_slow,delta,delta_rolling
1870,2025-12-05 15:25:00,45011.083431,45017.856475,45001.493464,45008.266509,235,2025-12-05,45009.205483,534679,2.404185e+10,44965.023573,45014.285058,44971.697067,-235.0,7794.0
1871,2025-12-05 15:26:00,45014.232637,45016.927540,45008.617273,45011.312176,1610,2025-12-05,45012.285663,536289,2.411432e+10,44965.165459,45015.165013,44973.683610,1610.0,7839.0
1872,2025-12-05 15:27:00,44994.657086,45003.083215,44987.043377,44995.469506,1992,2025-12-05,44995.198700,538281,2.420395e+10,44965.276602,45015.077594,44975.652693,-1992.0,3538.0
1873,2025-12-05 15:28:00,44997.152248,44999.573385,44993.470701,44995.891837,1934,2025-12-05,44996.311974,540215,2.429098e+10,44965.387711,45014.834352,44977.756375,1934.0,4497.0
1874,2025-12-05 15:29:00,45011.588146,45014.062617,45005.595167,45008.069638,1029,2025-12-05,45009.242474,541244,2.433729e+10,44965.471086,45015.186392,44979.922685,1029.0,3874.0


In [40]:
entry_cond = (
    (d["ma_fast"] > d["ma_slow"]) &
    (d["ma_fast"].shift(1) <= d["ma_slow"].shift(1)) &
    (d["close"] > d["vwap"]) &
    (d["delta_rolling"] > 0)
)

exit_cond = (
    (d["ma_fast"] < d["ma_slow"]) |
    (d["close"] < d["vwap"])
)

trades = []
in_pos = False
entry_price = None
entry_time = None

slippage_bps = 1.0
fee_bps = 0.5
cost_rate = (slippage_bps + fee_bps) / 10000

for i, row in d.dropna(subset=["ma_fast","ma_slow","vwap"]).iterrows():
    price = float(row["close"])
    ts = row["datetime"]

    if (not in_pos) and entry_cond.loc[i]:
        in_pos = True
        entry_price = price * (1 + cost_rate)
        entry_time = ts

    elif in_pos and exit_cond.loc[i]:
        exit_price = price * (1 - cost_rate)
        pnl = exit_price - entry_price
        ret = pnl / entry_price

        trades.append([entry_time, ts, entry_price, exit_price, pnl, ret])

        in_pos = False
        entry_price = None
        entry_time = None

trades_df = pd.DataFrame(trades, columns=["entry_time","exit_time","entry_price","exit_price","pnl","return"])
trades_df.head()


,entry_time,exit_time,entry_price,exit_price,pnl,return
0,2025-12-01 12:04:00,2025-12-01 12:06:00,44944.616011,44916.582628,-28.033384,-0.000624
1,2025-12-01 14:21:00,2025-12-01 15:09:00,45005.931250,45013.883190,7.951940,0.000177
2,2025-12-02 09:20:00,2025-12-02 09:29:00,45041.112795,45013.465906,-27.646889,-0.000614
3,2025-12-02 10:00:00,2025-12-02 10:42:00,45070.680037,45042.338422,-28.341614,-0.000629
4,2025-12-02 14:40:00,2025-12-02 15:25:00,45178.310016,45199.948047,21.638031,0.000479


In [41]:
# Create equity dataframe
equity = d[["datetime"]].copy()
equity["trade_return"] = 0.0

if not trades_df.empty:
    temp = trades_df.copy()
    temp["exit_time"] = pd.to_datetime(temp["exit_time"])

    # Loop approach (simple & safe)
    for _, row in temp.iterrows():
        exit_time = row["exit_time"]
        ret = row["return"]

        equity.loc[equity["datetime"] == exit_time, "trade_return"] += ret

# Equity curve
equity["equity"] = (1 + equity["trade_return"]).cumprod()

# Metrics
peak = equity["equity"].cummax()
drawdown = (equity["equity"] / peak) - 1
max_dd = drawdown.min()

rets = equity["equity"].pct_change().dropna()
sharpe = 0 if rets.std() == 0 else (np.sqrt(252) * (rets.mean() / rets.std()))

print("Trades:", len(trades_df))
print("Total Return %:", (equity["equity"].iloc[-1] - 1) * 100)
print("Max Drawdown %:", max_dd * 100)
print("Sharpe:", sharpe)


Trades: 10
Total Return %: -0.5389422606549621
Max Drawdown %: -0.5389422606549621
Sharpe: -0.8099827932628726


In [42]:
equity.head()

,datetime,trade_return,equity
0,2025-12-01 09:15:00,0.0,1.0
1,2025-12-01 09:16:00,0.0,1.0
2,2025-12-01 09:17:00,0.0,1.0
3,2025-12-01 09:18:00,0.0,1.0
4,2025-12-01 09:19:00,0.0,1.0


In [43]:
trades_df.to_csv("reports/trades.csv", index=False)

daily = equity.copy()
daily["date"] = pd.to_datetime(daily["datetime"]).dt.date
daily = daily.groupby("date")["equity"].agg(["first","last"])
daily["daily_return"] = (daily["last"]/daily["first"]) - 1
daily = daily.reset_index()

daily.to_csv("reports/daily_summary.csv", index=False)

with open("reports/eod_report.txt", "w") as f:
    f.write("=== End of Day Strategy Report ===\n\n")
    f.write(f"Trades: {len(trades_df)}\n")
    f.write(f"Total Return %: {(equity['equity'].iloc[-1]-1)*100:.2f}\n")
    f.write(f"Max Drawdown %: {max_dd*100:.2f}\n")
    f.write(f"Sharpe: {sharpe:.2f}\n")

print("✅ Saved reports in reports/ folder")


✅ Saved reports in reports/ folder


In [44]:
import pandas as pd
pd.read_csv("reports/daily_summary.csv").head()


,date,first,last,daily_return
0,2025-12-01,1.000000,0.999553,-0.000447
1,2025-12-02,0.999553,0.998789,-0.000764
2,2025-12-03,0.998789,0.998517,-0.000273
3,2025-12-04,0.998517,0.995278,-0.003244
4,2025-12-05,0.995278,0.994611,-0.000670


In [49]:
import pandas as pd
pd.read_csv("reports/daily_summary.csv").head()


,date,first,equity,daily_return
0,2025-12-01,1.000000,0.999553,-0.000447
1,2025-12-02,0.999553,0.998789,-0.000764
2,2025-12-03,0.998789,0.998517,-0.000273
3,2025-12-04,0.998517,0.995278,-0.003244
4,2025-12-05,0.995278,0.994611,-0.000670
